In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.impute import KNNImputer, SimpleImputer
pd.set_option('display.max_columns', 200)

## Read data

In [2]:
df = pd.read_feather("train_data.ftr")
# convert S2 to datatime
df.S_2 = pd.to_datetime(df.S_2)
print(df.shape)
df.head()

(5531451, 191)


,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,D_43,D_44,B_4,D_45,B_5,R_2,D_46,D_47,D_48,D_49,B_6,B_7,B_8,D_50,D_51,B_9,R_3,D_52,P_3,B_10,D_53,S_5,B_11,S_6,D_54,R_4,S_7,B_12,S_8,D_55,D_56,B_13,R_5,D_58,S_9,B_14,D_59,D_60,D_61,B_15,S_11,D_62,D_63,D_64,D_65,B_16,B_17,B_18,B_19,D_66,B_20,D_68,S_12,R_6,S_13,B_21,D_69,B_22,D_70,D_71,D_72,S_15,B_23,D_73,P_4,D_74,D_75,D_76,B_24,R_7,D_77,B_25,B_26,D_78,D_79,R_8,R_9,S_16,D_80,R_10,R_11,B_27,D_81,D_82,S_17,R_12,B_28,R_13,D_83,R_14,R_15,D_84,R_16,B_29,B_30,S_18,D_86,D_87,R_17,R_18,D_88,B_31,S_19,R_19,B_32,S_20,R_20,R_21,B_33,D_89,R_22,R_23,D_91,D_92,D_93,D_94,R_24,R_25,D_96,S_22,S_23,S_24,S_25,S_26,D_102,D_103,D_104,D_105,D_106,D_107,B_36,B_37,R_26,R_27,B_38,D_108,D_109,D_110,D_111,B_39,D_112,B_40,S_27,D_113,D_114,D_115,D_116,D_117,D_118,D_119,D_120,D_121,D_122,D_123,D_124,D_125,D_126,D_127,D_128,D_129,B_41,B_42,D_130,D_131,D_132,D_133,R_28,D_134,D_135,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-03-09,0.938477,0.001734,0.008728,1.006836,0.009224,0.124023,0.008774,0.004707,NaN,NaN,0.000630,0.080994,0.708984,0.170654,0.006203,0.358643,0.525391,0.255615,NaN,0.063904,0.059418,0.006466,0.148682,1.335938,0.008209,0.001423,0.207275,0.736328,0.096191,NaN,0.023376,0.002768,0.008324,1.001953,0.008301,0.161377,0.148315,0.922852,0.354492,0.151978,0.118103,0.001882,0.158569,0.065735,0.018387,0.063660,0.199585,0.308350,0.016357,0.401611,0.091064,CR,O,0.007126,0.007664,NaN,0.652832,0.008522,NaN,0.004730,6.0,0.271973,0.008362,0.515137,0.002644,0.009010,0.004807,0.008339,0.119385,0.004803,0.108276,0.050873,NaN,0.007553,0.080444,0.069092,NaN,0.004326,0.007561,NaN,0.007729,0.000272,0.001575,0.004238,0.001434,NaN,0.002272,0.004059,0.007122,0.002457,0.002310,0.003532,0.506836,0.008034,1.009766,0.084656,0.003820,0.007042,0.000438,0.006451,0.000830,0.005054,NaN,0.0,0.005722,0.007084,NaN,0.000198,0.008911,NaN,1.0,0.002537,0.005177,0.006626,0.009705,0.007782,0.002449,1.000977,0.002665,0.007481,0.006893,1.503906,1.005859,0.003569,0.008873,0.003948,0.003647,0.004951,0.894043,0.135620,0.911133,0.974609,0.001244,0.766602,1.008789,1.004883,0.893555,NaN,0.669922,0.009972,0.004570,NaN,1.008789,2.0,NaN,0.004326,NaN,NaN,NaN,1.007812,0.210083,0.676758,0.007874,1.0,0.238281,0.0,4.0,0.232178,0.236206,0.0,0.702148,0.434326,0.003057,0.686523,0.008743,1.0,1.002930,1.007812,1.000000,0.006805,NaN,0.002052,0.005970,NaN,0.004345,0.001534,NaN,NaN,NaN,NaN,NaN,0.002426,0.003706,0.003819,NaN,0.000569,0.000610,0.002674,0
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-04-07,0.936523,0.005775,0.004925,1.000977,0.006153,0.126709,0.000798,0.002714,NaN,NaN,0.002525,0.069397,0.712891,0.113220,0.006207,0.353516,0.521484,0.223389,NaN,0.065247,0.057739,0.001614,0.149780,1.339844,0.008369,0.001984,0.202759,0.720703,0.099792,NaN,0.030594,0.002748,0.002481,1.008789,0.005135,0.140991,0.143555,0.919434,0.326660,0.156250,0.118713,0.001610,0.148438,0.093933,0.013039,0.065491,0.151367,0.265137,0.017685,0.406250,0.086792,CR,O,0.002413,0.007149,NaN,0.646973,0.002237,NaN,0.003880,6.0,0.188965,0.004028,0.509277,0.004192,0.007843,0.001283,0.006523,0.140625,0.000094,0.101013,0.040466,NaN,0.004833,0.081421,0.074158,NaN,0.004204,0.005302,NaN,0.001864,0.000978,0.009895,0.007599,0.000509,NaN,0.009811,0.000126,0.005966,0.000395,0.001327,0.007774,0.500977,0.000761,1.009766,0.081848,0.000347,0.007790,0.004311,0.002333,0.009468,0.003754,NaN,0.0,0.007584,0.006676,NaN,0.001143,0.005905,NaN,1.0,0.008430,0.008980,0.001854,0.009926,0.005989,0.002247,1.006836,0.002508,0.006828,0.002836,1.503906,1.005859,0.000571,0.000391,0.008354,0.008850,0.003180,0.902344,0.136353,0.919922,0.975586,0.004562,0.786133,1.000000,1.003906,0.906738,NaN,0.668457,0.003922,0.004654,NaN,1.002930,2.0,NaN,0.008705,NaN,NaN,NaN,1.007812,0.184082,0.822266,0.003445,1.0,0.247192,0.0,4.0,0.243530,0.241943,0.0,0.707031,0.430420,0.001306,0.686523,0.000755,1.0,1.008789,1.003906,1.008789,0.004406,NaN,0.001034,0.

## Fill NA

In [4]:
# drop columns has more than 50% null
def na_col(df):
    del_cols = []
    for col in df.columns:
        if df[col].isna().sum() / df.shape[0] > 0.5:
            del_cols.append(col)
    return del_cols
del_cols = na_col(df)
print(f'delete {len(del_cols)} columns: {del_cols}')
df_reduced = df.drop(columns = del_cols)

delete 30 columns: ['D_42', 'D_49', 'D_50', 'D_53', 'D_56', 'S_9', 'B_17', 'D_66', 'D_73', 'D_76', 'R_9', 'D_82', 'B_29', 'D_87', 'D_88', 'D_105', 'D_106', 'R_26', 'D_108', 'D_110', 'D_111', 'B_39', 'B_42', 'D_132', 'D_134', 'D_135', 'D_136', 'D_137', 'D_138', 'D_142']


In [14]:
# impute categorical variables
cat_cols = ["B_30","B_38","D_114","D_116","D_117","D_120","D_126","D_63","D_64","D_68"]
np.sum(df_reduced[cat_cols].isnull())

/Users/lezhang/Documents/onecareer/onecareer-venv/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


B_30       2016
B_38       2016
D_114    176716
D_116    176716
D_117    176716
D_120    176716
D_126    116816
D_63          0
D_64          0
D_68     216503
dtype: int64

In [24]:
simple_imputer = SimpleImputer(missing_values=np.NaN, strategy='most_frequent')
data_simple_imputed = simple_imputer.fit_transform(df_reduced[cat_cols])
df_simple_imputed = pd.DataFrame(data_simple_imputed, columns = cat_cols)
np.sum(df_simple_imputed[cat_cols].isnull())

/Users/lezhang/Documents/onecareer/onecareer-venv/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


B_30     0
B_38     0
D_114    0
D_116    0
D_117    0
D_120    0
D_126    0
D_63     0
D_64     0
D_68     0
dtype: int64

In [29]:
# impute numerical variables
num_cols = df_reduced.columns[2:-1]
num_cols = [col for col in num_cols if col not in cat_cols]
print(f"impute {len(num_cols)} numerical columns")
imputer = SimpleImputer(missing_values=np.NaN, strategy='median')
data_imputed = imputer.fit_transform(df_reduced[num_cols])
df_numeric_imputed = pd.DataFrame(data_imputed, columns = num_cols)

impute 148 numerical columns


In [30]:
np.sum(df_numeric_imputed[num_cols].isnull())

/Users/lezhang/Documents/onecareer/onecareer-venv/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


P_2      0
D_39     0
B_1      0
B_2      0
R_1      0
        ..
D_140    0
D_141    0
D_143    0
D_144    0
D_145    0
Length: 148, dtype: int64

In [26]:
# impute categorical variables
# fill na with knnimputer
# # create an object for KNNImputer
# imputer = KNNImputer(n_neighbors=5)
# num_cols = df_reduced.columns[2:-1]
# num_cols = [col for col in num_cols if col not in cat_cols]
# print(f"impute {len(num_cols)} numerical columns")
# data_imputed = imputer.fit_transform(df_reduced[num_cols])
# df_numeric_imputed = pd.DataFrame(data_imputed, columns = num_cols)

In [32]:
df_imputed = pd.concat([df_reduced[['customer_ID','target']], df_simple_imputed, df_numeric_imputed], axis=1)
print(df_imputed.shape)
df_imputed.head(2)

(5531451, 160)


,customer_ID,target,B_30,B_38,D_114,D_116,D_117,D_120,D_126,D_63,D_64,D_68,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_43,D_44,B_4,D_45,B_5,R_2,D_46,D_47,D_48,B_6,B_7,B_8,D_51,B_9,R_3,D_52,P_3,B_10,S_5,B_11,S_6,D_54,R_4,S_7,B_12,S_8,D_55,B_13,R_5,D_58,B_14,D_59,D_60,D_61,B_15,S_11,D_62,D_65,B_16,B_18,B_19,B_20,S_12,R_6,S_13,B_21,D_69,B_22,D_70,D_71,D_72,S_15,B_23,P_4,D_74,D_75,B_24,R_7,D_77,B_25,B_26,D_78,D_79,R_8,S_16,D_80,R_10,R_11,B_27,D_81,S_17,R_12,B_28,R_13,D_83,R_14,R_15,D_84,R_16,S_18,D_86,R_17,R_18,B_31,S_19,R_19,B_32,S_20,R_20,R_21,B_33,D_89,R_22,R_23,D_91,D_92,D_93,D_94,R_24,R_25,D_96,S_22,S_23,S_24,S_25,S_26,D_102,D_103,D_104,D_107,B_36,B_37,R_27,D_109,D_112,B_40,S_27,D_113,D_115,D_118,D_119,D_121,D_122,D_123,D_124,D_125,D_127,D_128,D_129,B_41,D_130,D_131,D_133,R_28,D_139,D_140,D_141,D_143,D_144,D_145
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0,0.0,2.0,1.0,0.0,4.0,0.0,1.0,CR,O,6.0,0.938477,0.001734,0.008728,1.006836,0.009224,0.124023,0.008774,0.004707,0.088501,0.000630,0.080994,0.708984,0.170654,0.006203,0.358643,0.525391,0.255615,0.063904,0.059418,0.006466,1.335938,0.008209,0.001423,0.207275,0.736328,0.096191,0.023376,0.002768,0.008324,1.001953,0.008301,0.161377,0.148315,0.922852,0.354492,0.118103,0.001882,0.158569,0.018387,0.063660,0.199585,0.308350,0.016357,0.401611,0.091064,0.007126,0.007664,0.652832,0.008522,0.00473,0.271973,0.008362,0.515137,0.002644,0.009010,0.004807,0.008339,0.119385,0.004803,0.108276,0.050873,0.007553,0.080444,0.069092,0.004326,0.007561,0.205811,0.007729,0.000272,0.001575,0.004238,0.001434,0.002272,0.004059,0.007122,0.002457,0.002310,0.003532,0.008034,1.009766,0.084656,0.003820,0.007042,0.000438,0.006451,0.000830,0.005054,0.005722,0.007084,0.000198,0.008911,1.0,0.002537,0.005177,0.006626,0.009705,0.007782,0.002449,1.000977,0.002665,0.007481,0.006893,1.503906,1.005859,0.003569,0.008873,0.003948,0.003647,0.004951,0.894043,0.135620,0.911133,0.974609,0.001244,0.766602,1.008789,1.004883,0.669922,0.009972,0.004570,1.008789,0.004326,1.007812,0.210083,0.676758,0.007874,0.238281,0.232178,0.236206,0.702148,0.434326,0.003057,0.686523,0.008743,1.002930,1.007812,1.000000,0.006805,0.002052,0.005970,0.004345,0.001534,0.002426,0.003706,0.003819,0.000569,0.000610,0.002674
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0,0.0,2.0,1.0,0.0,4.0,0.0,1.0,CR,O,6.0,0.936523,0.005775,0.004925,1.000977,0.006153,0.126709,0.000798,0.002714,0.088501,0.002525,0.069397,0.712891,0.113220,0.006207,0.353516,0.521484,0.223389,0.065247,0.057739,0.001614,1.339844,0.008369,0.001984,0.202759,0.720703,0.099792,0.030594,0.002748,0.002481,1.008789,0.005135,0.140991,0.143555,0.919434,0.326660,0.118713,0.001610,0.148438,0.013039,0.065491,0.151367,0.265137,0.017685,0.406250,0.086792,0.002413,0.007149,0.646973,0.002237,0.00388,0.188965,0.004028,0.509277,0.004192,0.007843,0.001283,0.006523,0.140625,0.000094,0.101013,0.040466,0.004833,0.081421,0.074158,0.004204,0.005302,0.205811,0.001864,0.000978,0.009895,0.007599,0.000509,0.009811,0.000126,0.005966,0.000395,0.001327,0.007774,0.000761,1.009766,0.081848,0.000347,0.007790,0.004311,0.002333,0.009468,0.003754,0.007584,0.006676,0.001143,0.005905,1.0,0.008430,0.008980,0.001854,0.009926,0.005989,0.002247,1.006836,0.002508,0.006828,0.002836,1.503906,1.005859,0.000571,0.000391,0.008354,0.008850,0.003180,0.902344,0.136353,0.919922,0.975586,0.004562,0.786133,1.000000,1.003906,0.668457,0.003922,0.004654,1.002930,0.008705,1.007812,0.184082,0.822266,0.003445,0.247192,0.243530,0.241943,0.707031,0.430420,0.001306,0.686523,0.000755,1.008789,1.003906,1.008789,0.004406,0.001034,0.004837,0.007496,0.004932,0.003956,0.003166,0.005032,0.009575,0.005493,0.009216


## Aggregation

In [36]:
# agg numerical features by mean, std, min, max, last
num_agg_df = df_imputed.groupby("customer_ID")[num_cols].agg(['mean', 'std', 'min', 'max', 'last'])
num_agg_df.columns = ['_'.join(x) for x in num_agg_df.columns]
num_agg_df = num_agg_df.reset_index()

In [37]:
num_agg_df

,customer_ID,P_2_mean,P_2_std,P_2_min,P_2_max,P_2_last,D_39_mean,D_39_std,D_39_min,D_39_max,D_39_last,B_1_mean,B_1_std,B_1_min,B_1_max,B_1_last,B_2_mean,B_2_std,B_2_min,B_2_max,B_2_last,R_1_mean,R_1_std,R_1_min,R_1_max,R_1_last,S_3_mean,S_3_std,S_3_min,S_3_max,S_3_last,D_41_mean,D_41_std,D_41_min,D_41_max,D_41_last,B_3_mean,B_3_std,B_3_min,B_3_max,B_3_last,D_43_mean,D_43_std,D_43_min,D_43_max,D_43_last,D_44_mean,D_44_std,D_44_min,D_44_max,D_44_last,B_4_mean,B_4_std,B_4_min,B_4_max,B_4_last,D_45_mean,D_45_std,D_45_min,D_45_max,D_45_last,B_5_mean,B_5_std,B_5_min,B_5_max,B_5_last,R_2_mean,R_2_std,R_2_min,R_2_max,R_2_last,D_46_mean,D_46_std,D_46_min,D_46_max,D_46_last,D_47_mean,D_47_std,D_47_min,D_47_max,D_47_last,D_48_mean,D_48_std,D_48_min,D_48_max,D_48_last,B_6_mean,B_6_std,B_6_min,B_6_max,B_6_last,B_7_mean,B_7_std,B_7_min,B_7_max,B_7_last,B_8_mean,B_8_std,B_8_min,B_8_max,...,D_119_mean,D_119_std,D_119_min,D_119_max,D_119_last,D_121_mean,D_121_std,D_121_min,D_121_max,D_121_last,D_122_mean,D_122_std,D_122_min,D_122_max,D_122_last,D_123_mean,D_123_std,D_123_min,D_123_max,D_123_last,D_124_mean,D_124_std,D_124_min,D_124_max,D_124_last,D_125_mean,D_125_std,D_125_min,D_125_max,D_125_last,D_127_mean,D_127_std,D_127_min,D_127_max,D_127_last,D_128_mean,D_128_std,D_128_min,D_128_max,D_128_last,D_129_mean,D_129_std,D_129_min,D_129_max,D_129_last,B_41_mean,B_41_std,B_41_min,B_41_max,B_41_last,D_130_mean,D_130_std,D_130_min,D_130_max,D_130_last,D_131_mean,D_131_std,D_131_min,D_131_max,D_131_last,D_133_mean,D_133_std,D_133_min,D_133_max,D_133_last,R_28_mean,R_28_std,R_28_min,R_28_max,R_28_last,D_139_mean,D_139_std,D_139_min,D_139_max,D_139_last,D_140_mean,D_140_std,D_140_min,D_140_max,D_140_last,D_141_mean,D_141_std,D_141_min,D_141_max,D_141_last,D_143_mean,D_143_std,D_143_min,D_143_max,D_143_last,D_144_mean,D_144_std,D_144_min,D_144_max,D_144_last,D_145_mean,D_145_std,D_145_min,D_145_max,D_145_last
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.933782,0.024194,0.868652,0.960449,0.934570,0.010702,0.024440,0.001082,0.091492,0.009117,0.012008,0.006546,0.001930,0.021652,0.009384,1.005183,0.003248,1.000000,1.009766,1.007812,0.004509,0.003081,0.000263,0.009224,0.006104,0.113206,0.011665,0.098877,0.135010,0.135010,0.005021,0.003331,0.000685,0.009857,0.001604,0.006456,0.002942,0.000783,0.009865,0.007175,0.088501,0.000000,0.088501,0.088501,0.088501,0.004673,0.002823,0.000630,0.009399,0.003258,0.040308,0.031726,0.000836,0.080994,0.070801,0.725361,0.009530,0.708984,0.740234,0.740234,0.146644,0.047200,0.060486,0.231689,0.231689,0.005754,0.002928,0.001436,0.009918,0.008308,0.378042,0.085663,0.230957,0.519531,0.420410,0.532827,0.006533,0.521484,0.541992,0.539551,0.251399,0.069469,0.135620,0.403564,0.192383,0.113511,0.047368,0.063904,0.221924,0.149536,0.036624,0.023195,0.001681,0.060516,0.058411,0.003586,0.002080,0.000839,0.006466,...,0.244723,0.005258,0.236206,0.256592,0.256592,0.711877,0.005881,0.702148,0.719727,0.719727,0.433725,0.002515,0.429932,0.437500,0.433838,0.003103,0.002277,0.000428,0.009209,0.003580,0.686599,0.002747,0.682129,0.689941,0.685059,0.006254,0.003707,0.000097,0.009666,0.008400,1.006235,0.003076,1.001953,1.009766,1.008789,1.004132,0.003237,0.999512,1.007812,0.999512,1.005634,0.003040,1.000000,1.008789,1.008789,0.006753,0.002213,0.003222,0.009972,0.003975,0.005195,0.003004,0.001034,0.009850,0.004185,0.005446,0.002357,0.000715,0.008713,0.005703,0.004761,0.003116,0.000122,0.009224,0.006210,0.004089,0.002822,0.000139,0.009125,0.002716,0.003664,0.002616,0.000427,0.009743,0.007187,0.005342,0.002356,0.001976,0.009605,0.004234,0.005178,0.002608,0.000427,0.009384,0.005085,0.005066,0.003565,0.000569,0.009575,0.005810,0.005283,0.002598,0.000610,0.009613,0.002970,0.005813,0.003294,0.000996,0.009827,0.008530
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,0.899827,0.022097,0.861328,0.929199,0.880371,0.215171,0.199123,0.002224,0.567383,0.178101,0.025654,0.027750,0.006710,0.109619,0.034698,0.991098,0.051519,0.819824,1.008789,1.003906,0.0

In [38]:
# agg categorical features by count, last, unique
cat_agg_df = df_imputed.groupby("customer_ID")[cat_cols].agg(['count', 'last', 'nunique'])
cat_agg_df.columns = ['_'.join(x) for x in cat_agg_df.columns]
cat_agg_df = cat_agg_df.reset_index()

In [39]:
cat_agg_df

,customer_ID,B_30_count,B_30_last,B_30_nunique,B_38_count,B_38_last,B_38_nunique,D_114_count,D_114_last,D_114_nunique,D_116_count,D_116_last,D_116_nunique,D_117_count,D_117_last,D_117_nunique,D_120_count,D_120_last,D_120_nunique,D_126_count,D_126_last,D_126_nunique,D_63_count,D_63_last,D_63_nunique,D_64_count,D_64_last,D_64_nunique,D_68_count,D_68_last,D_68_nunique
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,13,0.0,1,13,2.0,1,13,1.0,1,13,0.0,1,13,4.0,1,13,0.0,1,13,1.0,1,13,CR,1,13,O,1,13,6.0,1
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,13,0.0,1,13,2.0,1,13,1.0,1,13,0.0,1,13,-1.0,1,13,0.0,2,13,1.0,1,13,CO,1,13,O,1,13,6.0,1
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,13,0.0,1,13,1.0,1,13,1.0,2,13,0.0,1,13,-1.0,1,13,0.0,1,13,1.0,1,13,CO,1,13,R,1,13,6.0,1
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,13,0.0,1,13,2.0,1,13,1.0,1,13,0.0,1,13,6.0,2,13,0.0,1,13,1.0,1,13,CO,1,13,O,1,13,3.0,3
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,13,0.0,1,13,1.0,2,13,1.0,1,13,0.0,1,13,4.0,1,13,0.0,1,13,1.0,1,13,CO,1,13,O,1,13,6.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458908,ffff41c8a52833b56430603969b9ca48d208e7c192c6a4...,13,0.0,1,13,2.0,1,13,0.0,1,13,0.0,1,13,3.0,2,13,1.0,2,13,1.0,1,13,CO,1,13,U,1,13,6.0,1
458909,ffff518bb2075e4816ee3fe9f3b152c57fc0e6f01bf7fd...,13,0.0,2,13,7.0,2,13,1.0,1,13,0.0,1,13,2.0,2,13,0.0,1,13,1.0,1,13,CO,1,13,R,1,13,6.0,1
458910,ffff9984b999fccb2b6127635ed0736dda94e544e67e02...,13,0.0,1,13,3.0,2,13,1.0,1,13,0.0,1,13,-1.0,1,13,0.0,1,13,1.0,1,13,CO,1,13,U,1,13,5.0,1
458911,ffffa5c46bc8de74f5a4554e74e239c8dee6b9baf38814...,13,0.0,1,13,3.0,1,13,1.0,2,13,0.0,1,13,2.0,2,13,0.0,1,13,1.0,1,13,CO,1,13,U,2,13,3.0,2


In [40]:
# read labels
df_label = pd.read_csv("train_labels.csv")
print(df_label.shape)
df_label.head()

(458913, 2)


,customer_ID,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,0
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,0
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,0
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,0


In [41]:
# merge features and labels
df_train = num_agg_df.merge(cat_agg_df, on = 'customer_ID').merge(df_label, on = 'customer_ID')
print(df_train.shape)
df_train.head()

(458913, 772)


,customer_ID,P_2_mean,P_2_std,P_2_min,P_2_max,P_2_last,D_39_mean,D_39_std,D_39_min,D_39_max,D_39_last,B_1_mean,B_1_std,B_1_min,B_1_max,B_1_last,B_2_mean,B_2_std,B_2_min,B_2_max,B_2_last,R_1_mean,R_1_std,R_1_min,R_1_max,R_1_last,S_3_mean,S_3_std,S_3_min,S_3_max,S_3_last,D_41_mean,D_41_std,D_41_min,D_41_max,D_41_last,B_3_mean,B_3_std,B_3_min,B_3_max,B_3_last,D_43_mean,D_43_std,D_43_min,D_43_max,D_43_last,D_44_mean,D_44_std,D_44_min,D_44_max,D_44_last,B_4_mean,B_4_std,B_4_min,B_4_max,B_4_last,D_45_mean,D_45_std,D_45_min,D_45_max,D_45_last,B_5_mean,B_5_std,B_5_min,B_5_max,B_5_last,R_2_mean,R_2_std,R_2_min,R_2_max,R_2_last,D_46_mean,D_46_std,D_46_min,D_46_max,D_46_last,D_47_mean,D_47_std,D_47_min,D_47_max,D_47_last,D_48_mean,D_48_std,D_48_min,D_48_max,D_48_last,B_6_mean,B_6_std,B_6_min,B_6_max,B_6_last,B_7_mean,B_7_std,B_7_min,B_7_max,B_7_last,B_8_mean,B_8_std,B_8_min,B_8_max,...,D_127_std,D_127_min,D_127_max,D_127_last,D_128_mean,D_128_std,D_128_min,D_128_max,D_128_last,D_129_mean,D_129_std,D_129_min,D_129_max,D_129_last,B_41_mean,B_41_std,B_41_min,B_41_max,B_41_last,D_130_mean,D_130_std,D_130_min,D_130_max,D_130_last,D_131_mean,D_131_std,D_131_min,D_131_max,D_131_last,D_133_mean,D_133_std,D_133_min,D_133_max,D_133_last,R_28_mean,R_28_std,R_28_min,R_28_max,R_28_last,D_139_mean,D_139_std,D_139_min,D_139_max,D_139_last,D_140_mean,D_140_std,D_140_min,D_140_max,D_140_last,D_141_mean,D_141_std,D_141_min,D_141_max,D_141_last,D_143_mean,D_143_std,D_143_min,D_143_max,D_143_last,D_144_mean,D_144_std,D_144_min,D_144_max,D_144_last,D_145_mean,D_145_std,D_145_min,D_145_max,D_145_last,B_30_count,B_30_last,B_30_nunique,B_38_count,B_38_last,B_38_nunique,D_114_count,D_114_last,D_114_nunique,D_116_count,D_116_last,D_116_nunique,D_117_count,D_117_last,D_117_nunique,D_120_count,D_120_last,D_120_nunique,D_126_count,D_126_last,D_126_nunique,D_63_count,D_63_last,D_63_nunique,D_64_count,D_64_last,D_64_nunique,D_68_count,D_68_last,D_68_nunique,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.933782,0.024194,0.868652,0.960449,0.934570,0.010702,0.024440,0.001082,0.091492,0.009117,0.012008,0.006546,0.001930,0.021652,0.009384,1.005183,0.003248,1.000000,1.009766,1.007812,0.004509,0.003081,0.000263,0.009224,0.006104,0.113206,0.011665,0.098877,0.135010,0.135010,0.005021,0.003331,0.000685,0.009857,0.001604,0.006456,0.002942,0.000783,0.009865,0.007175,0.088501,0.000000,0.088501,0.088501,0.088501,0.004673,0.002823,0.000630,0.009399,0.003258,0.040308,0.031726,0.000836,0.080994,0.070801,0.725361,0.009530,0.708984,0.740234,0.740234,0.146644,0.047200,0.060486,0.231689,0.231689,0.005754,0.002928,0.001436,0.009918,0.008308,0.378042,0.085663,0.230957,0.519531,0.420410,0.532827,0.006533,0.521484,0.541992,0.539551,0.251399,0.069469,0.135620,0.403564,0.192383,0.113511,0.047368,0.063904,0.221924,0.149536,0.036624,0.023195,0.001681,0.060516,0.058411,0.003586,0.002080,0.000839,0.006466,...,0.003076,1.001953,1.009766,1.008789,1.004132,0.003237,0.999512,1.007812,0.999512,1.005634,0.003040,1.000000,1.008789,1.008789,0.006753,0.002213,0.003222,0.009972,0.003975,0.005195,0.003004,0.001034,0.009850,0.004185,0.005446,0.002357,0.000715,0.008713,0.005703,0.004761,0.003116,0.000122,0.009224,0.006210,0.004089,0.002822,0.000139,0.009125,0.002716,0.003664,0.002616,0.000427,0.009743,0.007187,0.005342,0.002356,0.001976,0.009605,0.004234,0.005178,0.002608,0.000427,0.009384,0.005085,0.005066,0.003565,0.000569,0.009575,0.005810,0.005283,0.002598,0.000610,0.009613,0.002970,0.005813,0.003294,0.000996,0.009827,0.008530,13,0.0,1,13,2.0,1,13,1.0,1,13,0.0,1,13,4.0,1,13,0.0,1,13,1.0,1,13,CR,1,13,O,1,13,6.0,1,0
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,0.899827,0.022097,0.861328,0.929199,0.880371,0.215171,0.199123,0.002224,0.567383,0.178101,0.025654,0.027750,0.006710,0.109619,0.034698,0.991098,0.051519,0.819824,1.008789,1.003906,0.006246,0.002128,0.001023,0.008995,0.006912,0.120582,0.023842,0.089783,0.165527,0.165527,0.004993,0.002903,0.000728,0.009514,0.005550,0.005664,0.003355,0